## Rules
In salary cap contests, participants will create a lineup by selecting players listed in the Player Pool. Each player listed has an assigned salary and a valid lineup must not exceed the salary cap of $50,000.

Contest results will be determined by the total points accumulated by each individual lineup entry (scoring rules summarized below).

Participation in each contest must be made only as specified in the Terms of Use. Failure to comply with these Terms of Use will result in disqualification and, if applicable, prize forfeiture.

## Goals
Classic Game Types
Full Team Contests

In [143]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import floor

In [144]:
import pandas as pd
from random import sample

In [145]:
drop_attribs = [
        "game_date",
        'ppg_projection', 
        'value_projection', 
        'Proj FP', 
        'Proj Min', 
        'Proj Val',
        "position",
        "salary", 
]


## other drop attribs
"opp",
"spread",
"over_under",
"implied_team_score",
"L5_dvp_rank",
"L5_ppg_floor",
"L5_ppg_avg",
"L5_ppg_max",
"Rest",
"Opp Pace",
"Opp DEff",
"Opp DvP",
"L2 FGA",
"L5 FGA",
"S FGA",
"L2 Min",
"L5 Min",
"S Min",
"L5 FP",
"S FP",
"Floor FP",
"Ceil FP",


In [146]:
def reduceForProjectionAnalysis(file):
    file['FantasyFuelPPGProj'] = file['ppg_projection']
    file['FantasyFuelValueProj'] = file['value_projection']
    file['DFN_PPGProj'] = file['Proj FP']
    file['DFN_MinProj'] = file['Proj Min']
    file['DFN_ValueProj'] = file['Proj Val']
    file.drop(drop_attribs, axis=1, inplace=True)
    file = file.loc[file['Actual FP'] > 0]
    return file



In [147]:
##Helper Functions
def verifyLineup(lineup):
    enough_represented_teams = len(set(lineup['team'].to_list()))>=2
    under_salary_cap = lineup['Salary'].sum() <= 50000
    all_unique_players = len(set(lineup['Player Name'].to_list())) == 8
    if enough_represented_teams and under_salary_cap and all_unique_players:
        return True
    else:
        return False
    
def createRandomPopulation(point_guards, shooting_guards, small_forwards, power_forwards, guards, forwards, centers, util, limit):
    counter = 0
    lineups = []
    while(counter<limit):
        lineup = pd.DataFrame()
        lineup = lineup.append(point_guards.sample(n=1))
        lineup = lineup.append(shooting_guards.sample(n=1))
        lineup = lineup.append(small_forwards.sample(n=1))
        lineup = lineup.append(power_forwards.sample(n=1))
        lineup = lineup.append(guards.sample(n=1))
        lineup = lineup.append(forwards.sample(n=1))
        lineup = lineup.append(centers.sample(n=1))
        lineup = lineup.append(util.sample(n=1))
        if(verifyLineup(lineup)):
            lineups.append(lineup)
            counter = counter + 1
    return lineups


def mate(lineup1, lineup2):
    point_guards = lineup1.loc[lineup1['Pos'].str.contains("PG")].append(lineup2.loc[lineup2['Pos'].str.contains("PG")])
    shooting_guards = lineup1.loc[lineup1['Pos'].str.contains("SG")].append(lineup2.loc[lineup2['Pos'].str.contains("SG")])
    small_forwards = lineup1.loc[lineup1['Pos'].str.contains("SF")].append(lineup2.loc[lineup2['Pos'].str.contains("SF")])
    power_forwards = lineup1.loc[lineup1['Pos'].str.contains("PF")].append(lineup2.loc[lineup2['Pos'].str.contains("PF")])
    guards = lineup1.loc[lineup1['Pos'].str.contains("G")].append(lineup2.loc[lineup2['Pos'].str.contains("G")])
    forwards = lineup1.loc[lineup1['Pos'].str.contains("F")].append(lineup2.loc[lineup2['Pos'].str.contains("F")])
    centers = lineup1.loc[lineup1['Pos'].str.contains("C")].append(lineup2.loc[lineup2['Pos'].str.contains("C")])
    util = lineup1.append(lineup2)
    child_lineup = lineup1.append(lineup2)
    while(not verifyLineup(child_lineup)):
        child_lineup = pd.DataFrame()
        child_lineup = child_lineup.append(point_guards.sample(n=1))
        child_lineup = child_lineup.append(shooting_guards.sample(n=1))
        child_lineup = child_lineup.append(small_forwards.sample(n=1))
        child_lineup = child_lineup.append(power_forwards.sample(n=1))
        child_lineup = child_lineup.append(guards.sample(n=1))
        child_lineup = child_lineup.append(forwards.sample(n=1))
        child_lineup = child_lineup.append(centers.sample(n=1))
        child_lineup = child_lineup.append(util.sample(n=1))
    return child_lineup


def sortFitest(population): 
    population.sort(key=actualPointSum, reverse=True)
    population = eliminateDuplicates(population)
    return population 



def eliminateDuplicates(population):
    unique_lineups = set()
    unique_population = []
    for lineup in population:
        lineup_set = set()
        for player in lineup['Player Name'].tolist():
            lineup_set.add(player)
            
        if(not lineup_set.issubset(unique_lineups)):  
            unique_population.append(lineup)
            unique_lineups = unique_lineups.union(lineup_set)
    return unique_population


        

def actualPointSum(lineup):
    return lineup['Actual FP'].sum() 


In [148]:
def evaluateFitness(population):
    sum = 0 
    for lineup in population[:20]:
        sum = sum + lineup['Actual FP'].sum()
    return sum/20


def performSelection(population):
    selected_population = population[:1]
    selected_population.extend(sample(population[1:10], 5))
    selected_population.extend(sample(population[10:20], 4))
    selected_population.extend(sample(population, 9))
    #selected_population.extend(sample(population[50:], 10))
    return selected_population


def performCrossover(population):
    counter = 0
    children = []
    while(counter < 10):
        parents = sample(population,2)
        children.append(mate(parents[0],parents[1]))
        counter = counter+1
    return children
        

def createNextGeneration(old_population):
    selected_population = performSelection(old_population)
    children = performCrossover(selected_population)
    new_generation = old_population[:len(old_population)-10]
    new_generation.extend(children)
    return new_generation
    
        

In [149]:
def geneticAlgorithmForHistorialBestLineups(file):
    population_size = 50
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, population_size)  
    counter = 0
    while(counter<100):
        currentPopulation = sortFitest(population)
        population = createNextGeneration(currentPopulation)
        print(len(population))
        population.extend(createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 50 - len(population)))
        
        print(population[0]["Actual FP"].sum())
        counter = counter+1
    print("*****************")
    return sortFitest(population)

In [150]:
dates = [
    ['2_4', '2020-02-04'],
    ['2_3', '2020-02-03'],
    ['2_2', '2020-02-02'],
    ['2_1', '2020-02-01'],
    ['1_31', '2020-01-31'],
    ['1_30', '2020-01-30'],
    ['1_29', '2020-01-29'],
    ['1_28', '2020-01-28'],
    ['1_27', '2020-01-27'],
    ['1_26', '2020-01-26'],
    ['1_25', '2020-01-25'],
    ['1_24', '2020-01-24'],
    ['1_23', '2020-01-23'],
    ['1_22', '2020-01-22'],
    ['1_20', '2020-01-20'],
    ['1_19', '2020-01-19'],
    ['1_18', '2020-01-18'],
    ['1_17', '2020-01-17'],
    ['1_16', '2020-01-16'],
    ['1_15', '2020-01-15'],
    ['1_14', '2020-01-14'],
    ['1_13', '2020-01-13'],
    ['1_12', '2020-01-12'],
    ['1_11', '2020-01-11'],
    ['1_10', '2020-01-10'],
    ['1_9', '2020-01-09'],
    ['1_8', '2020-01-08'],
    ['1_7', '2020-01-07'],
    ['1_6', '2020-01-06'],
    ['1_5', '2020-01-05'],
    ['1_4', '2020-01-04'],
    ['1_3', '2020-01-03'],
    ['1_2', '2020-01-02'],
    ['1_1', '2020-01-01'],
    ['12_31', '2019-12-31'],
    ['12_30', '2019-12-30'],
    ['12_29', '2019-12-29'],
    ['12_28', '2019-12-28'],
    ['12_27', '2019-12-27'],
    ['12_26', '2019-12-26'],
    ['12_25', '2019-12-25'],
    ['12_23', '2019-12-23'],
    ['12_22', '2019-12-22'],
    ['12_21', '2019-12-21'],
    ['12_20', '2019-12-20'],
    ['12_19', '2019-12-19'],
    ['12_18', '2019-12-18'],
    ['12_17', '2019-12-17'],
    ['12_16', '2019-12-16'],
    ['12_15', '2019-12-15'],
    ['12_14', '2019-12-14'],
    ['12_13', '2019-12-13'],
    ['12_12', '2019-12-12'],
    ['12_11', '2019-12-11'],
    ['12_10', '2019-12-10'],
    ['12_9', '2019-12-09'],
    ['12_8', '2019-12-08'],
    ['12_7', '2019-12-07'],
    ['12_6', '2019-12-06'],
    ['12_5', '2019-12-05'],
    ['12_4', '2019-12-04'],
    ['12_3', '2019-12-03'],
    ['12_2', '2019-12-02'],
    ['12_1', '2019-12-01'],
    ['11_30', '2019-11-30'],
    ['11_29', '2019-11-29'],
    ['11_27', '2019-11-27'],
    ['11_26', '2019-11-26'],
    ['11_25', '2019-11-25'],
    ['11_24', '2019-11-24'],
    ['11_23', '2019-11-23'],
    ['11_22', '2019-11-22'],
    ['11_21', '2019-11-21'],
    ['11_20', '2019-11-20'],
    ['11_19', '2019-11-19'],
    ['11_18', '2019-11-18'],
    ['11_17', '2019-11-17'],
    ['11_16', '2019-11-16'],
    ['11_15', '2019-11-15'],
    ['11_14', '2019-11-14'],
    ['11_13', '2019-11-13'],
    ['11_12', '2019-11-12'],
    ['11_11', '2019-11-11'],
    ['11_10', '2019-11-10'],
    ['11_9', '2019-11-09'],
    ['11_8', '2019-11-08'],
    ['11_7', '2019-11-07'],
    ['11_6', '2019-11-06'],
    ['11_5', '2019-11-05'],
    ['11_4', '2019-11-04'],
    ['11_3', '2019-11-03'],
    ['11_2', '2019-11-02'],
    ['11_1', '2019-11-01'],
    ['10_31', '2019-10-31'],
    ['10_30', '2019-10-30'],
    ['10_29', '2019-10-29'],
    ['10_28', '2019-10-28'],
    ['10_27', '2019-10-27'],
    ['10_26', '2019-10-26'],
    ['10_25', '2019-10-25'],
    ['10_24', '2019-10-24'],
    ['10_23', '2019-10-23'],
    ['10_22', '2019-10-22'],   
]

def resetIndicies(file):
    for lineup in file:
        lineup.reset_index(drop=True, inplace=True)
    return file

def saveBestLineups_toCSV(file, date):
    key = np.arange(len(file))
    file_name = 'BestLineups/{}'.format(date)
    new_file = pd.concat(file, keys=key, names=['Lineup Num'])
    new_file.to_csv(file_name)

def createBestLineups():
    for game_day in dates:
        file_name = 'HistoricalData_Merged/{}'.format(game_day[1])
        cleaned_file = reduceForProjectionAnalysis(pd.read_csv(file_name))
        best_population = geneticAlgorithmForHistorialBestLineups(cleaned_file)
        saveBestLineups_toCSV(resetIndicies(best_population), game_day[1])
        

In [151]:
createBestLineups()

26
269.2
28
292.7
28
292.7
34
292.7
30
297.5
35
297.5
37
297.5
34
297.5
31
297.5
34
308.5
32
308.5
35
308.5
36
322.29999999999995
38
322.29999999999995
36
322.29999999999995
39
322.29999999999995
39
337.29999999999995
37
337.29999999999995
36
337.29999999999995
38
337.29999999999995
37
337.29999999999995
39
337.29999999999995
38
337.29999999999995
40
340.09999999999997
36
340.09999999999997
37
340.09999999999997
38
340.09999999999997
40
340.79999999999995
42
340.79999999999995
43
340.79999999999995
40
340.79999999999995
40
340.79999999999995
39
340.79999999999995
40
342.79999999999995
41
342.79999999999995
42
356.5
39
356.5
39
356.5
38
356.5
38
356.5
41
356.5
41
356.5
43
356.5
38
356.5
42
356.5
43
356.5
39
356.5
38
356.5
41
356.5
42
356.5
41
356.5
40
356.5
40
356.5
41
356.5
41
356.5
40
356.5
41
356.5
39
364.0
38
364.0
43
364.0
42
364.0
40
364.0
44
364.0
40
364.0
40
364.0
42
364.0
43
364.0
40
364.0
41
364.0
43
364.0
43
371.7
43
371.7
41
371.7
41
371.7
43
371.7
39
371.7
44
371.7
40
371.7

44
265.6
46
321.29999999999995
44
321.29999999999995
42
321.29999999999995
44
321.29999999999995
43
321.29999999999995
44
321.29999999999995
42
321.29999999999995
43
321.29999999999995
42
321.29999999999995
43
327.1
42
327.1
40
347.6
47
363.6
46
363.6
46
363.6
40
363.6
40
363.6
41
363.6
42
363.6
42
363.6
43
363.6
44
363.6
40
363.6
42
363.6
44
363.6
42
363.6
42
363.6
42
363.6
46
363.6
40
363.6
43
388.6
45
388.6
40
388.6
42
388.6
42
388.6
43
388.6
40
388.6
42
388.6
44
388.6
43
388.6
42
388.6
43
388.6
39
388.6
40
388.6
44
388.6
41
388.6
42
388.6
45
388.6
44
388.6
40
397.1
40
397.1
43
397.1
44
397.1
44
397.1
41
397.1
43
397.1
42
397.1
45
397.1
40
397.1
41
397.1
42
397.1
42
403.4
44
403.4
42
403.4
41
403.4
41
403.4
41
403.4
40
403.4
40
407.8
41
407.8
39
407.8
42
407.8
43
407.8
44
407.8
44
407.8
40
407.8
41
407.8
40
407.8
44
407.8
42
407.8
42
407.8
43
407.8
40
407.8
43
407.8
42
415.8
43
415.8
43
415.8
41
415.8
43
415.8
40
415.8
43
415.8
42
415.8
43
415.8
42
415.8
40
415.8
44
415.8
41
415.8
4

410.8
42
410.8
41
410.8
41
410.8
45
410.8
44
410.8
40
410.8
41
410.8
43
410.8
41
410.8
42
410.8
41
410.8
39
410.8
40
410.8
41
410.8
43
410.8
42
410.8
42
410.8
40
410.8
42
410.8
41
410.8
43
410.90000000000003
44
410.90000000000003
42
410.90000000000003
40
410.90000000000003
44
410.90000000000003
40
410.90000000000003
42
410.90000000000003
42
410.90000000000003
43
410.90000000000003
40
410.90000000000003
42
410.90000000000003
41
410.90000000000003
40
410.90000000000003
42
410.90000000000003
44
410.90000000000003
42
418.1
41
418.1
43
418.1
45
418.1
42
418.1
43
418.1
42
418.1
42
418.1
40
418.1
43
418.1
41
418.1
44
418.1
43
418.1
42
418.1
42
418.1
43
418.1
40
418.1
42
418.1
*****************
15
258.2
19
269.20000000000005
20
272.0
20
272.0
24
283.2
22
291.7
23
300.9
24
300.9
24
303.7
25
303.7
25
303.7
24
303.7
26
303.7
22
303.7
24
303.7
24
303.7
25
303.7
25
303.7
24
303.7
26
317.2
26
317.2
25
317.2
27
317.2
25
317.2
28
317.2
26
317.2
26
317.2
27
317.2
26
317.2
26
317.2
26
317.2
25
317.2
26


34
248.6
35
249.1
37
256.2
40
270.09999999999997
40
281.79999999999995
42
294.2
40
305.5
43
317.5
44
317.5
41
317.5
41
317.5
40
317.5
40
317.5
42
323.2
42
323.2
45
323.59999999999997
40
323.59999999999997
46
341.0
42
341.0
41
341.0
43
341.0
43
341.0
43
341.0
40
341.0
41
341.5
42
341.5
43
341.5
43
341.5
44
355.5
40
355.5
39
355.5
39
355.5
41
355.5
42
355.5
43
355.5
45
355.5
41
368.0
43
368.0
42
368.0
41
368.0
41
368.0
43
368.0
42
368.0
42
368.0
44
368.0
42
368.0
44
368.0
43
368.0
40
368.0
41
368.0
42
368.0
42
368.0
41
368.0
42
368.0
41
368.0
41
368.0
43
368.0
42
368.0
42
368.0
44
368.0
41
368.0
41
368.0
41
368.0
43
368.0
41
368.0
41
368.0
42
368.0
48
368.0
42
368.0
43
368.0
41
368.0
40
368.0
42
368.0
43
368.0
40
368.0
41
368.0
41
368.0
43
368.0
41
368.0
43
368.0
41
368.0
41
368.0
40
368.0
41
368.0
43
370.0
42
370.0
40
370.0
45
370.0
41
370.0
41
370.0
40
370.0
43
370.0
41
370.0
43
370.0
39
370.0
41
370.0
41
370.0
44
370.0
40
370.0
*****************
37
303.5
36
303.5
41
305.7
37
305.7
44


380.79999999999995
43
380.79999999999995
40
380.79999999999995
38
380.79999999999995
40
380.79999999999995
40
380.79999999999995
41
388.5
39
388.5
43
388.5
42
388.5
40
388.5
39
388.5
37
392.5
40
392.5
43
392.5
39
392.5
41
392.5
39
392.5
40
392.5
41
392.5
41
392.5
41
392.5
40
392.5
41
392.5
40
392.5
40
392.5
44
392.5
42
392.5
40
392.5
42
392.5
41
392.5
39
392.5
40
392.5
43
392.5
42
392.5
40
392.5
40
392.5
39
392.5
39
392.5
40
392.5
42
392.5
40
392.5
40
392.5
40
392.5
41
392.5
44
392.5
41
392.5
39
392.5
39
392.5
40
392.79999999999995
43
392.79999999999995
41
392.79999999999995
40
392.79999999999995
39
392.79999999999995
40
392.79999999999995
43
392.79999999999995
39
392.79999999999995
41
395.29999999999995
43
395.29999999999995
*****************
42
313.20000000000005
44
313.20000000000005
45
313.20000000000005
47
313.3
42
313.3
44
324.4
42
324.4
41
339.4
43
339.4
43
339.4
40
345.8
42
345.8
39
345.8
41
345.8
45
345.8
44
345.8
45
345.8
42
345.8
43
345.8
44
345.8
43
345.8
40
345.8
42
345.8


...

26
403.70000000000005
26
403.70000000000005
29
403.70000000000005
27
403.70000000000005
26
403.70000000000005
27
403.70000000000005
26
403.70000000000005
28
403.70000000000005
28
403.70000000000005
28
403.70000000000005
28
403.70000000000005
29
403.70000000000005
29
403.70000000000005
27
403.70000000000005
28
403.70000000000005
27
403.70000000000005
27
403.70000000000005
25
403.70000000000005
27
403.70000000000005
26
403.70000000000005
26
403.70000000000005
26
403.70000000000005
25
403.70000000000005
27
403.70000000000005
27
403.70000000000005
28
403.70000000000005
28
403.70000000000005
26
403.70000000000005
26
403.70000000000005
25
403.70000000000005
27
403.70000000000005
26
403.70000000000005
28
403.70000000000005
29
403.70000000000005
29
403.70000000000005
28
403.70000000000005
25
403.70000000000005
27
403.70000000000005
28
403.70000000000005
27
403.70000000000005
27
403.70000000000005
28
403.70000000000005
*****************
47
265.4
42
265.4
44
272.1
46
273.7
45
277.4
42
277.4
42
2

39
349.2
40
349.2
41
349.2
39
362.4
39
362.4
36
362.4
40
374.29999999999995
40
374.29999999999995
39
374.29999999999995
43
374.29999999999995
40
374.29999999999995
41
374.29999999999995
39
374.29999999999995
39
374.29999999999995
40
374.29999999999995
42
374.9
40
374.9
44
375.4
40
375.4
39
375.4
44
375.4
40
375.4
41
375.4
41
375.4
40
375.4
42
375.4
44
383.6
40
383.6
43
383.6
42
383.6
43
383.6
42
383.6
42
383.6
42
383.6
44
383.6
40
383.6
42
383.6
39
383.6
40
383.6
40
383.6
38
383.6
41
383.6
45
383.6
42
383.6
45
383.6
40
383.6
43
385.4
40
385.4
40
385.4
41
385.4
39
385.4
43
385.4
44
387.9
45
387.9
42
387.9
41
387.9
42
387.9
41
387.9
42
390.2
42
390.2
40
390.2
45
390.2
42
390.2
40
390.2
42
390.2
42
390.2
42
390.2
40
399.7
42
399.7
40
399.7
41
399.7
43
399.7
42
399.7
41
399.7
44
399.7
42
399.7
41
399.7
41
399.7
*****************
39
255.29999999999998
41
277.7
42
280.7
44
281.7
39
285.1
42
287.4
47
328.9
46
328.9
41
328.9
41
328.9
41
328.9
43
328.9
43
333.5
44
333.5
43
333.5
43
334.5
43
334

40
421.1
41
421.1
42
421.1
43
421.1
42
421.1
41
421.1
41
421.1
43
421.1
41
421.1
44
421.1
41
421.1
42
421.1
41
421.1
43
421.1
43
421.1
43
425.1
44
425.1
44
425.1
43
425.1
40
425.1
41
434.9
43
434.9
42
434.9
42
434.9
40
434.9
44
434.9
43
434.9
41
434.9
43
434.9
40
434.9
40
434.9
41
434.9
43
434.9
44
434.9
39
434.9
42
434.9
42
434.9
43
442.6
44
442.6
42
442.6
40
442.6
46
442.6
41
442.6
42
442.6
42
442.6
41
442.6
41
442.6
42
442.6
45
442.6
43
442.6
40
442.6
45
447.1
45
447.1
41
447.1
42
447.1
41
447.1
42
447.1
40
447.1
41
447.1
44
447.1
41
449.6
44
449.6
41
449.6
*****************
40
280.5
37
283.0
42
283.0
45
283.0
42
291.2
42
291.2
42
315.0
42
315.0
43
315.0
43
315.0
41
315.0
42
315.0
40
315.0
41
315.0
41
320.79999999999995
44
354.7
42
354.7
44
354.7
43
354.7
43
354.7
43
354.7
44
354.7
42
354.7
42
354.7
42
354.7
43
354.7
47
364.2
43
364.2
43
364.2
43
370.2
40
370.2
40
370.2
42
370.2
40
370.2
43
370.2
41
373.5
42
373.5
44
373.5
41
374.9
42
374.9
40
386.0
40
386.0
41
386.0
41
386.0
40
386

29
261.5
29
274.0
30
274.0
29
274.0
33
285.5
32
289.0
34
297.20000000000005
30
298.7
32
298.7
35
304.20000000000005
31
306.4
34
312.0
35
319.5
35
339.7
38
339.7
37
339.7
35
339.7
35
339.7
39
339.7
37
339.7
38
339.7
41
339.7
37
339.7
37
339.7
35
339.7
40
339.7
40
339.7
40
339.7
42
339.7
39
339.7
37
339.7
40
339.7
40
339.7
36
339.7
39
339.7
38
339.7
37
339.7
37
339.7
36
339.7
41
339.7
41
339.7
38
339.7
39
339.7
37
339.7
37
339.7
39
339.7
38
339.7
39
339.7
38
339.7
40
339.7
36
339.7
36
339.7
39
339.7
40
339.7
37
339.7
39
339.7
39
339.7
39
339.7
42
339.7
39
339.7
39
339.7
40
339.7
36
339.7
39
339.7
41
339.7
36
339.7
39
339.7
40
339.7
43
339.7
40
339.7
39
339.7
40
339.7
39
339.7
42
339.7
41
339.7
40
339.7
40
339.7
39
339.7
38
339.7
39
339.7
37
339.7
38
339.7
40
339.7
41
339.7
38
339.7
39
339.7
39
339.7
38
339.7
39
339.7
39
339.7
42
339.7
37
339.7
39
339.7
40
339.7
38
339.7
39
339.7
38
339.7
*****************
46
272.0
43
272.0
45
272.0
46
272.0
45
295.2
46
298.5
45
298.5
43
298.5
47
304.0
42

40
380.59999999999997
41
380.59999999999997
41
380.59999999999997
40
380.59999999999997
44
380.59999999999997
43
380.59999999999997
43
383.29999999999995
40
383.29999999999995
45
390.59999999999997
41
390.59999999999997
43
390.59999999999997
40
390.59999999999997
43
390.59999999999997
40
390.59999999999997
43
390.59999999999997
42
390.59999999999997
42
390.59999999999997
43
390.59999999999997
43
390.59999999999997
43
390.59999999999997
40
390.59999999999997
42
390.59999999999997
42
390.59999999999997
42
390.59999999999997
43
390.59999999999997
41
390.59999999999997
42
390.59999999999997
41
390.59999999999997
41
390.59999999999997
44
390.59999999999997
41
390.59999999999997
40
390.59999999999997
40
390.59999999999997
45
390.59999999999997
42
390.59999999999997
41
390.59999999999997
42
390.59999999999997
41
390.59999999999997
43
390.59999999999997
45
390.59999999999997
44
390.59999999999997
42
390.59999999999997
43
390.59999999999997
40
390.59999999999997
40
390.59999999999997
42
390.599

##main method 
#population_size = 100

def geneticAlgorithmForHistorialBestLineups(file):
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, population_size)
    prevGenFitness = -100
    currentGenFitness = 0
    
    while(currentGenFitness-prevGenFitness>1):
        currentPopulation = sortFitest(population)
        prevGenFitness = currentGenFitness
        currentGenFitness = evaluateFitness(currentPopulation)
        population = createNextGeneration(currentPopulation)
        population.extend(createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 10))
        print(population[0]["Actual FP"].sum())
    return currentPopulation

test = reduceForProjectionAnalysis(pd.read_csv('HistoricalData_Merged/2020-01-24'))
pop = geneticAlgorithmForHistorialBestLineups(test)



def eliminateDuplicates(population):
    unique_lineups = set()
    unique_population = []
    for lineup in population:
        if lineup['Actual FP'].sum() not in unique_lineups:
            unique_lineups.add(lineup['Actual FP'].sum())
            unique_population.append(lineup)
    return unique_population
    
def printDistib(population):
    h = []
    for x in population:
        print(x['Actual FP'].sum())
        h.append(x['Actual FP'].sum())

    std = np.std(h) 
    mean = np.mean(h)
    print(mean)
    print(std)
    print(mean-std)
    plt.plot(norm.pdf(h,mean,std))
    
    
test['Player Name']

se = set()
for lineup in pop2:
    for player in lineup['Player Name'].tolist():
        se.add(player)
    
len(list(se))

def createBestLineup():
    file_name = 'HistoricalData_Merged/2020-02-03'
    cleaned_file = reduceForProjectionAnalysis(pd.read_csv(file_name))
    best_population = geneticAlgorithmForHistorialBestLineups(cleaned_file)
    return best_population
    
def createBestLineup():
    file_name = 'HistoricalData_Merged/2020-02-03'
    cleaned_file = reduceForProjectionAnalysis(pd.read_csv(file_name))
    best_population = geneticAlgorithmForHistorialBestLineups(cleaned_file)
    return best_population